# <hr style="clear: both" />

# Running Spark in YARN-client mode

This notebook demonstrates how to set up a SparkContext that uses SURFsara's Hadoop cluster: [YARN resourcemanager](http://head05.hathi.surfsara.nl:8088/cluster) (note you will need to be authenticated via kerberos on your machine to visit the resourcemanager link) for executors.

First initialize kerberos via a Jupyter terminal. 
In the terminal execute: <BR>
<i>kinit -k -t data/robertop.keytab robertop@CUA.SURFSARA.NL</i><BR>
Print your credentials:


In [ ]:
! klist

In [ ]:
print sc

# <hr style="clear: both" />

# Now you can run your code

Pick a clustering algorithm (name of the file that provides a classify(x,y [,threshold]) function)

In [180]:
execfile('../spark-scripts/conventions.py')
execfile('../spark-scripts/eval.py')
execfile('../spark-scripts/implicitPlaylistAlgoFunctions.py')
execfile('../spark-scripts/implicitPlaylistAlgoMain.py')

CLUSTER_ALGO = 'jaccardBase'

execfile('../spark-scripts/' + CLUSTER_ALGO + '.py')


# Reading the conf file

In [177]:
import json
import copy

BASE_PATH = "/mnt/space/mattia"

conf = {}

conf['split'] = {}
conf['split']['reclistSize'] = 100
conf['split']['callParams'] = {}
conf['split']['excludeAlreadyListenedTest'] = True
conf['split']['name'] = 'SenzaRipetizioni_1'
conf['split']['split'] = conf['split']['name']
conf['split']['minEventsPerUser'] = 5
conf['split']['inputData'] = BASE_PATH + '/' + CLUSTER_ALGO + '.split/SenzaRipetizioni_1'
#conf['split']['inputData'] = 's3n://contentwise-research-poli/30musicdataset/newFormat/relations/sessions.idomaar'
conf['split']['bucketName'] = BASE_PATH
conf['split']['percUsTr'] = 0.05
conf['split']['ts'] = int(0.75 * (1421745857 - 1390209860) + 1390209860) - 10000
conf['split']['minEventPerSession'] = 5
conf['split']['onlineTrainingLength'] = 5
conf['split']['GTlength'] = 1
conf['split']['minEventPerSessionTraining'] = 10
conf['split']['minEventPerSessionTest'] = 11
conf['split']['mode'] = 'session'
conf['split']['forceSplitCreation'] = False
conf['split']["prop"] = {'reclistSize': conf['split']['reclistSize']}
conf['split']['type'] = None
conf['split']['out'] = BASE_PATH + '/' + CLUSTER_ALGO + '.split'
conf['split']['location'] = '30Mdataset/relations/sessions'

conf['evaluation'] = {}
conf['evaluation']['metric'] = {}
conf['evaluation']['metric']['type'] = 'recall'
conf['evaluation']['metric']['prop'] = {}
conf['evaluation']['metric']['prop']['N'] = [1,2,5,10,15,20,25,50,100]
conf['evaluation']['name'] = 'recall@N'

conf['general'] = {}
conf['general']['clientname'] = CLUSTER_ALGO + '.split'
conf['general']['bucketName'] = BASE_PATH
conf['general']['tracksPath'] = '30Mdataset/entities/tracks.idomaar.gz'

conf['algo'] = {}
conf['algo']['name'] = CLUSTER_ALGO
conf['algo']['props'] = {}
# ***** EXAMPLE OF CONFIGURATION *****#
conf['algo']['props']["sessionJaccardShrinkage"] = 5
conf['algo']['props']["clusterSimilarityThreshold"] = 0.1
conf['algo']['props']["expDecayFactor"] = 0.7
# ****** END EXAMPLE ****************#

# <hr style="clear: both" />
# Pick the list of songs ad create clusters


In [198]:
import json
import string

def my_replace_punct(x):
    ret = ""
    for i in x:
        if i == '+':
            ret += ' '
        else:
            ret += i
    return ret

tracksRDD = sc.textFile(BASE_PATH + '/30Mdataset/entities/tracks.idomaar.gz')
tracksRDD = tracksRDD.map(lambda x: x.split('\t')).map(lambda x: (x[1], json.loads(x[3])['name'].split('/') ) )
tracksRDD = tracksRDD.map(lambda x: (x[0], " ".join( (x[1][0], x[1][2]) ) ))
tracksRDD = tracksRDD.map(lambda x : (x[0], my_replace_punct(x[1])))
tracksRDD = tracksRDD.map(lambda x: (x[0], tokenize_song(x[1]), x[1]))


sampleRDD = tracksRDD.take(5000)
sampleRDD = sc.parallelize(sampleRDD)

tracksIdsRDD = tracksRDD.map(lambda x: (x[0], [x[0]]))

tracksIdsRDD.take(3)


[(u'0', [u'0']), (u'1', [u'1']), (u'2', [u'2'])]

# <hr style="clear: both" />

Reduce the quantity of data by building RDD {word -> songs}.
For each word keep only couples of songs that match.


In [ ]:
#Build an RDD with ('word' -> (id, name))
wordsRDD = sampleRDD.flatMap(lambda x: [(i, (x[0], x[2])) for i in x[1]] )
wordsRDD.take(3)

#Group by 'word' and keep only the ones with more then 1 song
wordsRDD = wordsRDD.groupByKey().mapValues(list).filter(lambda x: len(x[1]) > 1)

#Compute a cartesian product for each list of songs with a common word
def filtered_cartesian(x):
    equal_couples = set()
    for i in range(len(x[1])):
        a = x[1][i]
        id_a = x[1][i][0]
        name_a = x[1][i][1]
        
        for j in range(i):
            b = x[1][j]
            id_b = x[1][j][0]
            name_b = x[1][j][1]
            if id_a != id_b:
                if classify(name_a, name_b):
                    equal_couples.add((a,b))
                    
    return (x[0], tuple(equal_couples))

coupleRDD = wordsRDD.map(filtered_cartesian).filter(lambda x: len(x[1]) > 1)
coupleRDD.take(3)

# <hr style="clear: both" />
Flip the dataset and map each song to the couples it belongs to. 
Group by key and for each song you have a cluster!


In [ ]:
#Flatmap the list of couples
flattedCoupleRDD = coupleRDD.flatMap(lambda x: [i for i in x[1]])
#For each couple, for each song, yield song->couple
flattedCoupleRDD = flattedCoupleRDD.flatMap(lambda x: ((i[0], (x[0], x[1]) )for i in x) )


#Group by key (song). Each song has now one cluster
def merge_couples(x, y):
    return list(set(x) | set(y))

songClusterRDD = flattedCoupleRDD.reduceByKey(merge_couples).map(lambda x: (x[0], [i[0] for i in x[1]]))
songClusterRDD.take(30)

# <hr style="clear: both" />
Complete the clusters with all the other songs. 
Then we need to map cluster to songs to have new IDs.

In [199]:
#In this way we obtain a complete RDD with song -> group of songs
def reduce_to_biggest(x, y):
    bigger = x if len(x) > len(y) else y
    result = sorted(bigger)
    return result
           
unionRDD = songClusterRDD.union(tracksIdsRDD).reduceByKey(reduce_to_biggest)
unionRDD.take(100)


[(u'3187564', [u'3187564']),
 (u'839798', [u'839798']),
 (u'3265519', [u'3265519']),
 (u'4724052', [u'4724052']),
 (u'228052', [u'228052']),
 (u'186098', [u'186098']),
 (u'1767625', [u'1767625']),
 (u'397389', [u'397389']),
 (u'2653961', [u'2653961']),
 (u'125577', [u'125577']),
 (u'2353162', [u'2353162']),
 (u'684237', [u'684237']),
 (u'3623366', [u'3623366']),
 (u'1570601', [u'1570601']),
 (u'1971070', [u'1971070']),
 (u'2103417', [u'2103417']),
 (u'3205031', [u'3205031']),
 (u'4707213', [u'4707213']),
 (u'1837140', [u'1837140']),
 (u'1251968', [u'1251968']),
 (u'2413655', [u'2413655']),
 (u'3601843', [u'3601843']),
 (u'1902200', [u'1902200']),
 (u'868912', [u'868912']),
 (u'3276442', [u'3276442']),
 (u'776530', [u'776530']),
 (u'3251955', [u'3251955']),
 (u'486721', [u'486721']),
 (u'1879897', [u'1879897']),
 (u'1751011', [u'1751011']),
 (u'127470', [u'127470']),
 (u'2816593', [u'2816593']),
 (u'916015', [u'916015']),
 (u'2141778', [u'2141778']),
 (u'704359', [u'704359']),
 (u'49834

# <hr style="clear: both" />
We have song -> cluster. We map inversly (cluster -> song) and group by key (cluster).
Finally we zip with index and obtain new IDs.

In [200]:
#Flip the mapping as cluster->song
clusterSongsRDD = unionRDD.map(lambda x: (' '.join(x[1]), x[0])).groupByKey().mapValues(list)
clusterSongsRDD = clusterSongsRDD.zipWithIndex().map(lambda x: (x[1], x[0][1]))
clusterSongsRDD.take(30)

[(0, [u'839798']),
 (1, [u'3265519']),
 (2, [u'4724052']),
 (3, [u'172037']),
 (4, [u'228052']),
 (5, [u'186098']),
 (6, [u'1741586']),
 (7, [u'397389']),
 (8, [u'2653961']),
 (9, [u'4972098']),
 (10, [u'931855']),
 (11, [u'684237']),
 (12, [u'2294826']),
 (13, [u'1570601']),
 (14, [u'1971070']),
 (15, [u'2103417']),
 (16, [u'1837140']),
 (17, [u'2413655']),
 (18, [u'3601843']),
 (19, [u'1902200']),
 (20, [u'868912']),
 (21, [u'3276442']),
 (22, [u'776530']),
 (23, [u'3251955']),
 (24, [u'1879897']),
 (25, [u'1751011']),
 (26, [u'127470']),
 (27, [u'2901109']),
 (28, [u'2141778']),
 (29, [u'704359'])]

In [203]:
clusterSongsRDD.count()

5675143

# <hr style="clear: both" />
Save the RDD (newID, songs) in "/mnt/space/mattia/clusters/[ALGORITHM]"

In [205]:
#Save clustering
clusterSongsRDD.saveAsTextFile(BASE_PATH + '/clusters/' + CLUSTER_ALGO)

Py4JJavaError: An error occurred while calling o7586.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/mnt/space/mattia/clusters/jaccardBase already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:132)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1179)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1156)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1156)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:150)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:111)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:316)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1156)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$

Create RDD with song -> clusterID

In [208]:
clusterSongsRDD = sc.textFile(BASE_PATH + '/clusters/' + CLUSTER_ALGO)
songToClusterRDD = clusterSongsRDD.flatMap(lambda x: ((i, x[0]) for i in x[1]) )
songToClusterRDD.cache()
songToClusterRDD.lookup(str(656564))[0]

2872622


Split the data

In [185]:
execfile('../spark-scripts/splitCluster2.py')
splitter(conf)

It's time to substitute songs with their cluster

In [ ]:
import json
execfile('../spark-scripts/utilsCluster2.py')
train, test = loadDataset(conf)
train.cache()
test.cache()

def flat_map_tracks_ids(x):
    objects = x['linkedinfo']['objects']
    result = []
    for i in range(len(objects)):
        result.append( (str(objects[i]['id']), i, x) )
    return result
        
for t in [train, test]:   
    t = t.map(lambda x: json.loads(x)).flatMap(flat_map_tracks_ids)
    t = t.join(songToClusterRDD)
    t.take(3)

In [ ]:
from os import path
basePath = path.join(conf['general']['bucketName'], conf['general']['clientname'])
splitPath = path.join(basePath, conf['split']['name'])

clusterSimList = [0.1]
sessionJaccardShrinkageList = [5]
expDecayList = [0.7]

for exclude in [True]:
    conf['split']['excludeAlreadyListenedTest'] = str(exclude)
    #conf['split']['name'] = 'giroCompletoTestMultipleConfs_exclude%s' % exclude
    #splitter(conf)
    #train, test = loadDataset(conf)
    #train.cache()
    #test.cache()
    
    for sessionJaccardShrinkage in sessionJaccardShrinkageList:
        conf['algo']['props']["sessionJaccardShrinkage"] = sessionJaccardShrinkage
        
        for clusterSim in clusterSimList:
            conf['algo']['props']["clusterSimilarityThreshold"] = clusterSim
            
            playlists = extractImplicitPlaylists(train, conf).cache()
            
            for expDecay in expDecayList:
                conf['algo']['props']["expDecayFactor"] = expDecay
                conf['algo']['name'] = CLUSTER_ALGO + '_ImplicitPlaylist_shk_%d_clustSim_%.3f_decay_%.3f' % \
                    (sessionJaccardShrinkage, clusterSim, expDecay )

                recJsonRDD = executeImplicitPlaylistAlgo(playlists, test, conf)
                try:
                    saveRecommendations(conf, recJsonRDD, overwrite=True)
                    try:
                        computeMetrics(conf)
                    except:
                        print 'Error in computing metrics'
                except:
                    print 'Error in saving recommndations'
                    try:
                        computeMetrics(conf)
                    except:
                        print 'Error in computing metrics'